<a href="https://colab.research.google.com/github/WrathOfXLord/LocalPyhtonWork/blob/main/mobile_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ali Fırat Özel - 18120205038

Kendi drive'ımda bulunan kddcup99 verisini çekebilmem için gerekli kod bloğu

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
column_labels = [
    "duration",
    "protocol_type",
    "service",
    "flag",
    "src_bytes",
    "dst_bytes",
    "land",
    "wrong_fragment",
    "urgent",
    "hot",
    "num_failed_logins",
    "logged_in",
    "num_compromised",
    "root_shell",
    "su_attempted",
    "num_root",
    "num_file_creations",
    "num_shells",
    "num_access_files",
    "num_outbound_cmds",
    "is_host_login",
    "is_guest_login",
    "count",
    "srv_count",
    "serror_rate",
    "srv_serror_rate",
    "rerror_rate",
    "srv_rerror_rate",
    "same_srv_rate",
    "diff_srv_rate",
    "srv_diff_host_rate",
    "dst_host_count",
    "dst_host_srv_count",
    "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate",
    "dst_host_srv_serror_rate",
    "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate",
    "outcome",
]

In [ ]:
# gerekli ve gerekebilecek bazı kütüphaneler
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
#for colab

id = 'https://drive.google.com/file/d/1pSWkf8NRqFelVFlZ_41XX2GUrliinkP7/view?usp=sharing'.split('/')[-2]
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('kddcup.data.corrected')
df = pd.read_csv('kddcup.data.corrected', sep=",", names=column_labels, header=None).sample(frac=0.15, random_state=10)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 734765 entries, 2908929 to 2264145
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     734765 non-null  int64  
 1   protocol_type                734765 non-null  object 
 2   service                      734765 non-null  object 
 3   flag                         734765 non-null  object 
 4   src_bytes                    734765 non-null  int64  
 5   dst_bytes                    734765 non-null  int64  
 6   land                         734765 non-null  int64  
 7   wrong_fragment               734765 non-null  int64  
 8   urgent                       734765 non-null  int64  
 9   hot                          734765 non-null  int64  
 10  num_failed_logins            734765 non-null  int64  
 11  logged_in                    734765 non-null  int64  
 12  num_compromised              734765 non-null  int64

verileri saldırı türlerine göre alfabetik sıraladık

In [ ]:
df.sort_values(by=[df.columns[-1]], ascending=True, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.00,0.17,1.0,0.0,0.33,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.
1,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.25,0.40,1.0,0.0,0.40,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.
2,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.05,0.05,back.
3,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.04,0.04,back.
4,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.20,0.20,1.0,0.0,0.00,135,135,1.00,0.00,0.01,0.00,0.00,0.00,0.05,0.05,back.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734760,0,tcp,ftp,SF,1242,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,32,0.13,0.02,0.00,0.00,0.09,0.00,0.00,0.00,warezclient.
734761,0,tcp,ftp_data,SF,334,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,1,9,1.00,0.00,1.00,0.44,0.00,0.00,0.00,0.00,warezclient.
734762,9,tcp,ftp_data,SF,0,5149533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,14,14,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,warezmaster.
734763,10,tcp,ftp_data,SF,0,5150823,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,7,7,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,warezmaster.


verileri saldırı türlerine göre numaralandırdık


In [ ]:
df["type"] = np.NaN
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,type
0,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.00,0.17,1.0,0.0,0.33,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.,NaN
1,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.25,0.40,1.0,0.0,0.40,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.,NaN
2,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.05,0.05,back.,NaN
3,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.04,0.04,back.,NaN
4,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.20,0.20,1.0,0.0,0.00,135,135,1.00,0.00,0.01,0.00,0.00,0.00,0.05,0.05,back.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734760,0,tcp,ftp,SF,1242,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,32,0.13,0.02,0.00,0.00,0.09,0.00,0.00,0.00,warezclient.,NaN
734761,0,tcp,ftp_data,SF,334,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,1,9,1.00,0.00,1.00,0.44,0.00,0.00,0.00,0.00,warezclient.,NaN
734762,9,tcp,ftp_data,SF,0,5149533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,14,14,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,warezmaster.,NaN
734763,10,tcp,ftp_data,SF,0,5150823,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,7,7,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,warezmaster.,NaN


In [ ]:
df.loc[df["outcome"] != "normal.", "type"] = "1"
df.loc[df["outcome"] == "normal.", "type"] = "0"


In [ ]:
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,type
0,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.00,0.17,1.0,0.0,0.33,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.,1
1,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.25,0.40,1.0,0.0,0.40,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.,1
2,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.05,0.05,back.,1
3,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.04,0.04,back.,1
4,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.20,0.20,1.0,0.0,0.00,135,135,1.00,0.00,0.01,0.00,0.00,0.00,0.05,0.05,back.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734760,0,tcp,ftp,SF,1242,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,32,0.13,0.02,0.00,0.00,0.09,0.00,0.00,0.00,warezclient.,1
734761,0,tcp,ftp_data,SF,334,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,1,9,1.00,0.00,1.00,0.44,0.00,0.00,0.00,0.00,warezclient.,1
734762,9,tcp,ftp_data,SF,0,5149533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,14,14,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,warezmaster.,1
734763,10,tcp,ftp_data,SF,0,5150823,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,7,7,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,warezmaster.,1


en çok bulunan ilk 10 tipi aldık (normal'de dahil)

In [ ]:
df["outcome"].value_counts()

smurf.              421463
neptune.            160931
normal.             145489
satan.                2414
ipsweep.              1878
portsweep.            1565
back.                  341
nmap.                  323
warezclient.           140
teardrop.              137
pod.                    52
guess_passwd.            7
land.                    6
buffer_overflow.         5
rootkit.                 4
imap.                    3
warezmaster.             3
multihop.                2
ftp_write.               1
loadmodule.              1
Name: outcome, dtype: int64

In [ ]:
attack_type = df.groupby("outcome").size().sort_values(ascending=False)
attack_type = pd.DataFrame(attack_type)
unwanted_types = attack_type[10:].index.values

for rows in range(len(unwanted_types)):
  df.drop(df[df.outcome == unwanted_types[rows]].index, inplace=True)

df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome,type
0,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,6,0.0,0.0,0.00,0.17,1.0,0.0,0.33,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.,1
1,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,4,5,0.0,0.0,0.25,0.40,1.0,0.0,0.40,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05,back.,1
2,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.05,0.05,back.,1
3,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,255,1.00,0.00,0.00,0.00,0.01,0.01,0.04,0.04,back.,1
4,0,tcp,http,SF,54540,8314,0,0,0,2,0,1,1,0,0,0,0,0,0,0,0,0,5,5,0.0,0.0,0.20,0.20,1.0,0.0,0.00,135,135,1.00,0.00,0.01,0.00,0.00,0.00,0.05,0.05,back.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734757,0,tcp,ftp_data,SF,334,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,3,3,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,warezclient.,1
734758,0,tcp,ftp_data,SF,334,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,8,26,0.88,0.25,1.00,0.12,0.00,0.00,0.00,0.00,warezclient.,1
734759,1,tcp,ftp,SF,1264,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,138,0.54,0.02,0.00,0.00,0.00,0.00,0.00,0.00,warezclient.,1
734760,0,tcp,ftp,SF,1242,2451,0,0,0,28,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255,32,0.13,0.02,0.00,0.00,0.09,0.00,0.00,0.00,warezclient.,1


istenen tipleri belirledik

In [ ]:
featureLabels = ["duration", 
                 "src_bytes", 
                 "dst_bytes", 
                 "land", 
                 "wrong_fragment", 
                 "urgent", 
                 "hot",
                 "num_failed_logins", 
                 "logged_in", 
                 "num_compromised", 
                 "root_shell", 
                 "su_attempted", 
                 "num_root", 
                 "num_file_creations", 
                 "num_shells",
                 "num_access_files", 
                 "num_outbound_cmds", 
                 "is_host_login", 
                 "is_guest_login", 
                 "count", 
                 "srv_count", 
                 "serror_rate", 
                 "srv_serror_rate", 
                 "rerror_rate", 
                 "srv_rerror_rate", 
                 "same_srv_rate", 
                 "diff_srv_rate", 
                 "srv_diff_host_rate",
                 "dst_host_count", 
                 "dst_host_srv_count", 
                 "dst_host_same_srv_rate", 
                 "dst_host_diff_srv_rate", 
                 "dst_host_same_src_port_rate", 
                 "dst_host_srv_diff_host_rate", 
                 "dst_host_serror_rate", 
                 "dst_host_srv_serror_rate",
                 "dst_host_rerror_rate", 
                 "dst_host_srv_rerror_rate"]
featureList = df[featureLabels].astype(float)

wanted_types = pd.get_dummies(df["outcome"]).columns.values

wanted_types


array(['back.', 'ipsweep.', 'neptune.', 'nmap.', 'normal.', 'portsweep.',
       'satan.', 'smurf.', 'teardrop.', 'warezclient.'], dtype=object)

test ve train datalarını belli yüzdelerle ayrıştırdık

In [ ]:
featureList_test = df.loc[df["outcome"] == wanted_types[0]].sample(frac=0.15, random_state=10)
featureList_train = df.loc[df["outcome"] == wanted_types[0]]

for typeIndex in range(1, len(wanted_types)):
  type_test = df.loc[df["outcome"] == wanted_types[typeIndex]].sample(frac=0.15, random_state=10)
  type_train = df.loc[df["outcome"] == wanted_types[typeIndex]].drop(type_test.index)
  featureList_test = pd.concat([featureList_test, type_test])
  featureList_train = pd.concat([featureList_train, type_train])
  

test ve train label'larını ayrıştırdık

In [ ]:
featureList_test.reset_index(drop=True, inplace=True)
featureList_train.reset_index(drop=True, inplace=True)

featureListTrain_labels = featureList_train.loc[:, ["outcome"]]
featureListTest_labels = featureList_test.loc[:, ["outcome"]]
featureListTrain_labels

,outcome
0,back.
1,back.
2,back.
3,back.
4,back.
...,...
624525,warezclient.
624526,warezclient.
624527,warezclient.
624528,warezclient.


In [ ]:
featureList_test = featureList_test[featureLabels].astype(float)
featureList_train = featureList_train[featureLabels].astype(float)
featureList_test

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.00,0.0,0.0,1.0,0.0,0.00,79.0,79.0,1.0,0.0,0.01,0.00,0.00,0.00,0.04,0.04
1,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.00,0.0,0.0,1.0,0.0,0.00,255.0,255.0,1.0,0.0,0.00,0.00,0.01,0.01,0.05,0.05
2,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,8.0,0.0,0.00,0.0,0.0,1.0,0.0,0.25,58.0,58.0,1.0,0.0,0.02,0.00,0.02,0.02,0.02,0.02
3,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.25,0.0,0.0,1.0,0.0,0.50,43.0,43.0,1.0,0.0,0.02,0.00,0.02,0.02,0.02,0.02
4,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.00,0.0,0.0,1.0,0.0,0.00,255.0,255.0,1.0,0.0,0.00,0.00,0.00,0.00,0.02,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110197,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.00,0.0,0.0,1.0,0.0,0.00,3.0,9.0,1.0,0.0,1.00,0.22,0.00,0.00,0.00,0.00
110198,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.00,0.0,0.0,1.0,0.0,0.00,5.0,38.0,0.6,0.4,0.60,0.16,0.00,0.00,0.00,0.00
110199,12.0,7045.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.00,0.0,0.0,1.0,0.0,0.00,15.0,42.0,1.0,0.0,1.00,0.07,0.00,0.00,0.00,0.00
110200,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.00,0.0,0.0,1.0,0.0,0.00,6.0,40.0,1.0,0.0,1.00,0.05,0.00,0.00,0.00,0.00


In [ ]:
featureList_train

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,6.0,0.0,0.0,0.00,0.17,1.0,0.0,0.33,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05
1,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.25,0.40,1.0,0.0,0.40,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.05,0.05
2,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255.0,255.0,1.00,0.00,0.00,0.00,0.01,0.01,0.05,0.05
3,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255.0,255.0,1.00,0.00,0.00,0.00,0.01,0.01,0.04,0.04
4,0.0,54540.0,8314.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.20,0.20,1.0,0.0,0.00,135.0,135.0,1.00,0.00,0.01,0.00,0.00,0.00,0.05,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624525,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00,0.00,1.0,0.0,0.00,1.0,35.0,1.00,0.00,1.00,0.14,0.00,0.00,0.00,0.00
624526,0.0,334.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.00,0.00,1.0,0.0,0.00,3.0,3.0,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00
624527,1.0,1264.0,2451.0,0.0,0.0,0.0,28.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255.0,138.0,0.54,0.02,0.00,0.00,0.00,0.00,0.00,0.00
624528,0.0,1242.0,2451.0,0.0,0.0,0.0,28.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.00,0.00,1.0,0.0,0.00,255.0,32.0,0.13,0.02,0.00,0.00,0.09,0.00,0.00,0.00


In [ ]:
featureListTrain_labels

,outcome
0,back.
1,back.
2,back.
3,back.
4,back.
...,...
624525,warezclient.
624526,warezclient.
624527,warezclient.
624528,warezclient.


train datamızı modele fit ettirdik

In [ ]:
import time as t
defaultVal = 5
clf = KNeighborsClassifier(n_neighbors = defaultVal, algorithm = 'ball_tree', leaf_size=500)
time_point = t.time()
clf.fit(featureList_train, featureListTrain_labels)
timedelta = t.time() - time_point
print("Classifier trained in {} seconds".format(round(timedelta, 4)))

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Classifier trained in 5.0578 seconds


tahmin datamızı test ediyoruz

In [ ]:
time_point = t.time()
predicted = clf.predict(featureList_test)
timedelta = t.time() - time_point
print("Classifier predicted results in {} seconds".format(round(timedelta, 4)))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


Classifier predicted results in 1696.1587 seconds


tahmin sonuçları

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(predicted, featureListTest_labels)
print("Accuracy: ", str(round(accuracy, 5) * 100) + "%")

Accuracy:  99.861%


In [ ]:
matrix = confusion_matrix(featureListTest_labels, predicted)
matrix = pd.DataFrame(matrix, index=wanted_types, columns=wanted_types)
matrix.index.name = "Attack Types"
matrix

,back.,ipsweep.,neptune.,nmap.,normal.,portsweep.,satan.,smurf.,teardrop.,warezclient.
Attack Types,,,,,,,,,,
back.,50,0,0,0,1,0,0,0,0,0
ipsweep.,0,269,0,12,1,0,0,0,0,0
neptune.,0,0,24130,0,5,1,4,0,0,0
nmap.,0,17,0,24,3,4,0,0,0,0
normal.,0,7,2,1,21804,3,0,5,0,1
portsweep.,0,2,11,1,10,201,10,0,0,0
satan.,0,0,41,0,3,2,316,0,0,0
smurf.,0,0,0,0,0,0,0,63219,0,0
teardrop.,0,0,1,0,4,0,0,0,16,0
